# Reading data from Wikipedia


In [1]:
import pandas as pd

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.columns=['Postal Code','Borough','Neighborhood']
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


# Clean and process the data

The following code do the following: (1)delete rows with 'Not Assigned' data in the column Borough, (2)assign Borough name in null data in  column Neighborhood, and(3) combine Neighborhood data in rows with similar Postal Code.


In [2]:
df = df[df.Borough != 'Not assigned']
df['Neighborhood'].loc[df['Neighborhood'].isnull()] =  df.Borough
df = (df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(list)).reset_index()
df

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,[Malvern / Rouge]
1,M1C,Scarborough,[Rouge Hill / Port Union / Highland Creek]
2,M1E,Scarborough,[Guildwood / Morningside / West Hill]
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
...,...,...,...
98,M9N,York,[Weston]
99,M9P,Etobicoke,[Westmount]
100,M9R,Etobicoke,[Kingsview Village / St. Phillips / Martin Gro...
101,M9V,Etobicoke,[South Steeles / Silverstone / Humbergate / Ja...


# Read CSV file of the coordinates of Postal Codes

In [3]:
geoinfo= pd.read_csv("http://cocl.us/Geospatial_data")
geoinfo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Add coordinates data (Latitude and Longitude) to the processed Wikipedia data

In [4]:
df_final=pd.merge(df,geoinfo,on='Postal Code',how='left' )
df_final

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,[Malvern / Rouge],43.806686,-79.194353
1,M1C,Scarborough,[Rouge Hill / Port Union / Highland Creek],43.784535,-79.160497
2,M1E,Scarborough,[Guildwood / Morningside / West Hill],43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,[Weston],43.706876,-79.518188
99,M9P,Etobicoke,[Westmount],43.696319,-79.532242
100,M9R,Etobicoke,[Kingsview Village / St. Phillips / Martin Gro...,43.688905,-79.554724
101,M9V,Etobicoke,[South Steeles / Silverstone / Humbergate / Ja...,43.739416,-79.588437


# Import necessary Libraries

In [5]:
import numpy as np
from sklearn.cluster import KMeans

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Create a map showing the location of postal codes in boroughs

Getting the coordinates of Toronto, Ontario:

In [6]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto,Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto,Canada are 43.6534817, -79.3839347.


In [7]:
# create map of Toronto, Ontario using latitude and longitude values
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postalcode, borough in zip(df_final['Latitude'],  df_final['Longitude'], df_final['Postal Code'], df_final['Borough']):
    label = '{},{}'.format (postalcode ,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Using Foursquare API to explore and segment boroughs

Defining Foursquare credentials and version:

In [8]:
CLIENT_ID = 'GVMXFDQXIWBMDLZAVK15PI1IP3LPDQ41VO53KYNO1YWNBOBM' # your Foursquare ID
CLIENT_SECRET = 'QIQ3LJHUV0LMPTPES1PMQW0QIE1OV1S21KAVCQSGPH3SQPLV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GVMXFDQXIWBMDLZAVK15PI1IP3LPDQ41VO53KYNO1YWNBOBM
CLIENT_SECRET:QIQ3LJHUV0LMPTPES1PMQW0QIE1OV1S21KAVCQSGPH3SQPLV


# Limiting data to those with Toronto in their borough names (Old Toronto area)

In [9]:
toronto_data = df_final[df_final['Borough'].str.contains('Toronto', regex=False)].reset_index(drop=True)
toronto_data.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,[The Beaches],43.676357,-79.293031
1,M4K,East Toronto,[The Danforth West / Riverdale],43.679557,-79.352188
2,M4L,East Toronto,[India Bazaar / The Beaches West],43.668999,-79.315572
3,M4M,East Toronto,[Studio District],43.659526,-79.340923
4,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790


Exploring the first borough in the dataframe:

In [10]:
toronto_data.loc[0,'Borough']

'East Toronto'

Determining first borough's coordinates:

In [11]:
borough_latitude = toronto_data.loc[0, 'Latitude'] # borough latitude value
borough_longitude = toronto_data.loc[0, 'Longitude'] # borough longitude value

borough_name = toronto_data.loc[0, 'Borough'] # borough name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of East Toronto are 43.67635739999999, -79.2930312.


Getting top 100 venues that are in East Toronto within the radius of 500:

In [12]:

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=GVMXFDQXIWBMDLZAVK15PI1IP3LPDQ41VO53KYNO1YWNBOBM&client_secret=QIQ3LJHUV0LMPTPES1PMQW0QIE1OV1S21KAVCQSGPH3SQPLV&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [13]:
results = requests.get(url).json()

Defining nand running a function that extracts the category of the venue:

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/michelle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


# Eploring boroughs in Toronto

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=toronto_data['Borough'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
Downtown Toronto
East Toronto


Size of resulting dataframe:

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(1623, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,East Toronto,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


Number of venues per borough:

In [20]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,114,114,114,114,114,114
Downtown Toronto,1225,1225,1225,1225,1225,1225
East Toronto,126,126,126,126,126,126
West Toronto,158,158,158,158,158,158


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


# Analyzing each borough

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move borough column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [23]:
toronto_onehot.shape

(1623, 232)

In [24]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.008772,0.000000,0.008772,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772,0.008772,0.000000,0.000000,0.008772,0.000000,0.017544,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.017544,0.000000,0.070175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.008772,0.035088,0.017544,0.000

In [25]:
toronto_grouped.shape

(4, 232)

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
              venue  freq
0       Coffee Shop  0.07
1              Park  0.06
2    Sandwich Place  0.06
3              Café  0.05
4  Sushi Restaurant  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2  Japanese Restaurant  0.03
3           Restaurant  0.03
4                Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.06
1         Coffee Shop  0.06
2             Brewery  0.04
3  Italian Restaurant  0.04
4                Café  0.04


----West Toronto----
                venue  freq
0                Café  0.07
1                 Bar  0.06
2  Italian Restaurant  0.04
3         Coffee Shop  0.04
4          Restaurant  0.03




In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
2,East Toronto,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
3,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Vegetarian / Vegan Restaurant,Bookstore


# Examining clusters of venues in the Old Toronto district

In [29]:
import numpy as np
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 1, 2], dtype=int32)

In [30]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data
toronto_merged.head()

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,[The Beaches],43.676357,-79.293031,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
1,M4K,East Toronto,[The Danforth West / Riverdale],43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
2,M4L,East Toronto,[India Bazaar / The Beaches West],43.668999,-79.315572,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
3,M4M,East Toronto,[Studio District],43.659526,-79.340923,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
4,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Defining the cluster

CLUSTER 1 - Venues in the Central Toronto which are predominantly parks, eating venues like coffee shop and sandwich shops, and gyms

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
5,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
6,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
7,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
8,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
9,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
22,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
23,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub
24,Central Toronto,0,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Restaurant,Pub


CLUSTER 2 -  East Toronto area characterized ny many European-influenced venues like Greek and Italian restaurants

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
1,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
2,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
3,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant
38,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant


CLUSTER 3 - West Toronto is a cluster of venues where you can find the the most vegetarian restaurants and bookstores

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,2,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Vegetarian / Vegan Restaurant,Bookstore
32,West Toronto,2,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Vegetarian / Vegan Restaurant,Bookstore
33,West Toronto,2,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Vegetarian / Vegan Restaurant,Bookstore
34,West Toronto,2,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Vegetarian / Vegan Restaurant,Bookstore
35,West Toronto,2,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Vegetarian / Vegan Restaurant,Bookstore
36,West Toronto,2,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Vegetarian / Vegan Restaurant,Bookstore


CLUSTER 3- Downtown Toronto venues that tends to business or tourist customers

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
11,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
12,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
13,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
14,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
15,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
16,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
17,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
18,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
19,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
